In [ ]:
# 기본 MNIST(multinomial classification)
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Data Loading
train_data = pd.read_csv("./data/kaggle/train.csv")
train_x_data = train_data.drop('label', axis = 1)
train_y_data = tf.one_hot(train_data["label"], depth=10).eval(session = tf.Session())
test_x_data = pd.read_csv("./data/kaggle/test.csv")
# display(tf.one_hot(train_data["label"], depth=10))
# print("-----")
# display(train_y_data)

# Tensorflow Graph Initialization
tf.reset_default_graph()
X = tf.placeholder(shape = [None, 784], dtype = tf.float32)
Y = tf.placeholder(shape = [None, 10], dtype = tf.float32)
keep_prob = tf.placeholder(dtype=tf.float32)

# Weight & bias
W1 = tf.get_variable("weight1", shape = [784,256], initializer=tf.contrib.layers.xavier_initializer())
b1 = tf.Variable(tf.random_normal([256]), name = "bias1")
_layer1 = tf.nn.relu(tf.matmul(X,W1)+b1)
layer1 = tf.nn.dropout(_layer1, keep_prob = keep_prob)

W2 = tf.get_variable("weight2", shape = [256,256], initializer=tf.contrib.layers.xavier_initializer())
b2 = tf.Variable(tf.random_normal([256]), name = "bias2")
_layer2 = tf.nn.relu(tf.matmul(layer1,W2)+b2)
layer2 = tf.nn.dropout(_layer2, keep_prob = keep_prob)

W3 = tf.get_variable("weight3", shape = [256,10], initializer=tf.contrib.layers.xavier_initializer())
b3 = tf.Variable(tf.random_normal([10]), name = "bias3")

# Hypothesis
logits = tf.matmul(layer2,W3) + b3
H = tf.nn.relu(logits)

# cost function
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = logits, labels = Y))

# train node
train = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)

# session object & initialization
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# epoch & batch size
training_epoch = 10
batch_size = 100

# training
for step in range(training_epoch):
    num_of_iteration = int(train_data.shape[0] / batch_size)
    cost_val = 0
    
    for i in range(num_of_iteration):
        batch_x, batch_y = train_x_data[i*batch_size:(i+1)*batch_size],train_y_data[i*batch_size:(i+1)*batch_size]
        _, cost_val = sess.run([train, cost], feed_dict={X: batch_x, Y: batch_y, keep_prob: 1.0})

    if step %5 == 0:
        print(cost_val)
        
#predict check
predict = tf.argmax(H,1)
result = sess.run(predict, feed_dict={X:test_x_data, keep_prob: 1.0})
df = pd.DataFrame({
    'ImageId': [i for i in range(1,28001)],
    'Label': result
})
df.to_csv('./data/kaggle/submission.csv', index=False)

correct = tf.equal(predict, tf.math.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct, dtype=tf.float32))
print("Accuracy: {}".format(sess.run(accuracy, feed_dict = {X: train_x_data, Y: train_y_data, keep_prob: 1.0})))

In [37]:
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

warnings.filterwarnings(action="ignore")




df = pd.read_csv("./data/titanic/train.csv", sep="," )
#display(df)
#데이터정제

df.drop(["Name","Ticket","Cabin"], axis=1, inplace=True)
df.dropna(how="any", inplace=True)
# NaN이 있는 모든 row 삭제

df.loc[df["Sex"]=="male","Sex"] = 1 #남자는 1, 여자는 2
df.loc[df["Sex"]=="female","Sex"] = 2
df.loc[:,"Age"] = (df["Age"]//10) #연령을 처리하기 쉽도록 0~9세는 0, 10~19세는 1, 20~29는 2… 식으로 표현
df.loc[df["Embarked"]=="S","Embarked"] = 1 # Embarked == 1: Southampton
df.loc[df["Embarked"]=="C","Embarked"] = 2 # Embarked == 2: Cherbourg
df.loc[df["Embarked"]=="Q","Embarked"] = 3 # Embarked == 3: Queenstown
df["Fare"]=pd.qcut(df["Fare"], 5, labels=[1,2,3,4,5])
display(df)


# x데이터 추출
df_x=df.drop("Survived", axis=1, inplace=False)
# # y데이터 추출
df_y = df["Survived"]

#training data set (Matrix형태 ) 
train_x_data = MinMaxScaler().fit_transform(df_x.values)
#display(train_x_data)
y_value = df_y.values
train_y_data = MinMaxScaler().fit_transform(y_value.reshape(-1,1))



# Tensorflow Graph Initialization
tf.reset_default_graph()


#placeholder
X=tf.placeholder(shape=[None,8], dtype=tf.float32)
Y=tf.placeholder(shape=[None,1], dtype=tf.float32)
keep_prob = tf.placeholder(dtype=tf.float32)

# Weight & bias

# Weight & bias
#W=tf.Variable(tf.random_normal([6,1]), name="weight")
#b=tf.Variable(tf.random_normal([1]), name="bias")

W1 = tf.get_variable("weight1", shape = [8,256], initializer=tf.contrib.layers.xavier_initializer())
b1 = tf.get_variable("bias1", shape = [256], initializer=tf.contrib.layers.xavier_initializer())
_layer1 = tf.nn.relu(tf.matmul(X,W1)+b1)
layer1 = tf.nn.dropout(_layer1, keep_prob = keep_prob)

W2 = tf.get_variable("weight2", shape = [256,256], initializer=tf.contrib.layers.xavier_initializer())
b2 = tf.get_variable("bias2", shape = [256], initializer=tf.contrib.layers.xavier_initializer())
_layer2 = tf.nn.relu(tf.matmul(layer1,W2)+b2)
layer2 = tf.nn.dropout(_layer2, keep_prob = keep_prob)

W3 = tf.get_variable("weight3", shape = [256,1], initializer=tf.contrib.layers.xavier_initializer())
b3 = tf.get_variable("bias3", shape = [1], initializer=tf.contrib.layers.xavier_initializer())




# Hypothesis
logits = tf.matmul(layer2,W3) + b3
H = tf.nn.relu(logits)

#cost function
cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits, labels=Y)) ##cost함수 Logistic function 변수 2개.

#training node생성
train = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)

#Session. & 초기화
sess=tf.Session()
sess.run(tf.global_variables_initializer())

# epoch & batch size
training_epoch = 100


#학습
for step in range(training_epoch):
    num_of_iteration = train_x_data.shape[0]
    cost_val=0
    
    for i in range(num_of_iteration):       
        _, cost_val = sess.run([train, cost], feed_dict={X: train_x_data, Y: train_y_data, keep_prob: 1.0})
    if step %10 == 0:
        print(cost_val)  

        

#Accuracy 측정
predict = tf.cast(H>0.5, dtype=tf.float32)
accuracy =tf.reduce_mean(tf.cast(tf.equal(predict,Y), dtype=tf.float32))
print("정확도:{}".format(sess.run(accuracy, feed_dict={X:train_x_data,Y:train_y_data, keep_prob: 1.0})))



,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,1,2.0,1,0,1,1
1,2,1,1,2,3.0,1,0,5,2
2,3,1,3,2,2.0,0,0,2,1
3,4,1,1,2,3.0,1,0,5,1
4,5,0,3,1,3.0,0,0,2,1
6,7,0,1,1,5.0,0,0,5,1
7,8,0,3,1,0.0,3,1,3,1
8,9,1,3,2,2.0,0,2,2,1
9,10,1,2,2,1.0,1,0,4,2
10,11,1,3,2,0.0,1,1,3,1


0.14182882
0.03611818
0.023519363
0.024530122
0.018907087
0.017146852
0.01284156
0.012360115
0.011377888
0.008018404
정확도:0.9915730357170105


In [44]:
#predict check

df2 = pd.read_csv("./data/titanic/test.csv", sep="," ,skiprows=0)
df2.drop(["Name","Ticket","Cabin"], axis=1, inplace=True)
#df2.dropna(how="any", inplace=True)

df2.loc[df2["Sex"]=="male","Sex"] = 1 #남자는 1, 여자는 2
df2.loc[df2["Sex"]=="female","Sex"] = 2
df2.loc[:,"Age"] = (df2["Age"]//10) #연령을 처리하기 쉽도록 0~9세는 0, 10~19세는 1, 20~29는 2… 식으로 표현
df2.loc[df2["Embarked"]=="S","Embarked"] = 1 # Embarked == 1: Southampton
df2.loc[df2["Embarked"]=="C","Embarked"] = 2 # Embarked == 2: Cherbourg
df2.loc[df2["Embarked"]=="Q","Embarked"] = 3 # Embarked == 3: Queenstown
df2["Fare"]=pd.qcut(df2["Fare"], 5, labels=[1,2,3,4,5])
display(df2)

df_ids=df2[["PassengerId"]]
df_test=df2
test_x_data = MinMaxScaler().fit_transform(df_test.values)
#display(train_x_data)
# y_value = df_y.values
# train_y_data = MinMaxScaler().fit_transform(y_value.reshape(-1,1))

#predict = tf.argmax(H,1)
predict=tf.cast(H>0.5, dtype=tf.float32) #예측값
result = sess.run(predict, feed_dict={X:test_x_data, keep_prob: 1.0})

df2 = pd.DataFrame({
    'PassengerId': [str(i[0]) for i in df_ids.values],
    'Survived': [int(i[0]) for i in result]
})
df2.to_csv('./data/titanic/submission.csv', index=False)


,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,892,3,1,3.0,0,0,2,3
1,893,3,2,4.0,1,0,1,1
2,894,2,1,6.0,0,0,2,3
3,895,3,1,2.0,0,0,2,1
4,896,3,2,2.0,1,1,3,1
5,897,3,1,1.0,0,0,2,1
6,898,3,2,3.0,0,0,1,3
7,899,2,1,2.0,1,1,4,1
8,900,3,2,1.0,0,0,1,2
9,901,3,1,2.0,2,0,4,1
